In [2]:
import os

%pwd

'c:\\Users\\Farouk\\Desktop\\TextSummarizer\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Farouk\\Desktop\\TextSummarizer'

In [4]:
### update config.yaml 


In [5]:
# entity 

from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy : str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
#configuration.py

from src.TextSummarizer.constants import CONFIG_FILE_PATH , PARAMS_FILE_PATH
from src.TextSummarizer.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_strategy =params.eval_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
            
            
        )
        return model_trainer_config 

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, PegasusForConditionalGeneration ,AutoModelForSeq2SeqLM
import torch
from datasets import load_from_disk

c:\Users\Farouk\Desktop\TextSummarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
    
    def train(self):
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt= load_from_disk(self.config.data_path) 
        
        trainer_args = TrainingArguments(
                            output_dir=self.config.root_dir,
                            num_train_epochs=self.params.num_train_epochs,
                            warmup_steps=self.params.warmup_steps,
                            per_device_train_batch_size=self.params.per_device_train_batch_size,
                            per_device_eval_batch_size=self.params.per_device_eval_batch_size,
                            weight_decay=self.params.weight_decay,
                            logging_steps=self.params.logging_steps,
                            eval_strategy =self.params.eval_strategy,
                            eval_steps=self.params.eval_steps,
                            save_steps=self.params.save_steps,
                            gradient_accumulation_steps=self.params.gradient_accumulation_steps
                            )
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["train"],
                  eval_dataset = dataset_samsum_pt["validation"])

        trainer.train()
        
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
device

'cuda'

In [1]:
import torch
print(torch.cuda.is_available())   # True if GPU is accessible
print(torch.cuda.get_device_name(0))  # GPU model name


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
# config = ConfigurationManager()
# model_trainer_config = config.get_model_trainer_config()
# model_trainer = ModelTrainer(config=model_trainer_config)
# model_trainer.train()